<a href="https://colab.research.google.com/github/gortegasolis/WDPA_geometric_attributes/blob/main/Koppen_climate_wdpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Authors: Gabriel Ortega

# This script is organized in four sections:
# 1. SETTINGS: Contains information about the input data and some pre-processing steps.
# 2. FUNCTION(S): Main function(s) to apply.
# 3. EXECUTION: Apply the function(s) to the input data.
# 4. EXPORT RESULTS: Export outside of EarthEngine.

import ee
ee.Authenticate()
ee.Initialize(project='gortega-research')

############################################################################################################################
# 1. SETTINGS

# Polygons collection
table = ee.FeatureCollection("projects/gortega-research/assets/wdpa_workdata")  # Explicitly define WDPA polygons
# Unique polygons identifier
uniqueID = 'wdpaid'

# Climate Dataset
climateDataset = ee.Image('projects/gortega-research/assets/koppen_geiger_0p01')  # Koppen-Geiger climate zones
climateBand = 'b1'  # Band name for climate classification

# Create a bounding box if necessary

# Original bounding box coordinates
xmin, ymin, xmax, ymax = -180, 90, 180, -90  # Full bounding box

############################################################################################################################
# 2. FUNCTION(S)
def processData():
  def calculateStats(feature):
        combinedGeometry = feature.geometry()

        # Get the climate data.
        climate = climateDataset.select(climateBand)

        # Use a histogram to get the pixel counts for each climate category within the combined polygon.
        climateStats = climate.reduceRegion(
            reducer=ee.Reducer.histogram(),
            geometry=combinedGeometry,
            scale=100,
            maxPixels=1e13
        )

        # Get the total number of pixels in the combined polygon.
        totalPixels = climate.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=combinedGeometry,
            scale=100,
            maxPixels=1e13
        ).get(climateBand)

        results = {
            'climate_histogram': climateStats.get(climateBand),
            'total_pixels': totalPixels
        }

        return ee.Feature(None, results).set(uniqueID, feature.get(uniqueID))

  features = filteredPols.map(calculateStats)
  return ee.FeatureCollection(features)

  ############################################################################################################################
  # 3. EXECUTION

# Loop over whichBox from 1 to 16
for whichBox in range(1, 17):
  # Initialize bbox variable
  if whichBox == 0:
    # Full bounding box
    bbox = ee.Geometry.BBox(xmin, ymin, xmax, ymax)
  else:
    # Grid configuration (4x4)
    xSegments, ySegments = 4, 4

    # Calculate step sizes
    xStep = (xmax - xmin) / xSegments  # 45° per column
    yStep = (ymax - ymin) / ySegments  # -22.5° per row

    # Convert whichBox (1-16) to grid indices
    adjustedIndex = whichBox - 1
    row = adjustedIndex // xSegments  # 0-3 (bottom to top)
    col = adjustedIndex % xSegments  # 0-3 (left to right)

    # Reverse row order (original grid is top to bottom)
    yIndex = (ySegments - 1) - row

    # Calculate coordinates
    xStart = xmin + col * xStep
    xEnd = xStart + xStep
    yStart = ymin + yIndex * yStep
    yEnd = yStart + yStep

    bbox = ee.Geometry.BBox(xStart, yStart, xEnd, yEnd)

  # Name of output file
  fileName = f'climate_analysis_WDPA_{xmin}_{ymin}_{xmax}_{ymax}_bbox{whichBox}'

  # Filter polygons dataset as required
  filteredPols = table.filterBounds(bbox)

  # Apply function
  allFeatures = processData()

  ############################################################################################################################
  # 4. EXPORT RESULTS
  task = ee.batch.Export.table.toDrive(
    collection=allFeatures,
    description=fileName,
    fileFormat='CSV',
    folder='Earthengine_exports'
  )
  task.start()